In [2]:
using Distributions
using ReactiveMP
using Rx
using BenchmarkTools

import Base: show

In [3]:
function createSubgraph(index::Int)
    noise_add = addition_node("[$index] noise_add", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);
    noise_node     = gaussian_mean_variance("[$index] noise_node");
    noise_mean     = constant_variable("[$index] noise_mean", 0.0, noise_node.mean);
    noise_variance = constant_variable("[$index] noise_variance", 1.0, noise_node.variance);
    
    tmp = random_variable("[$index] tmp", noise_node.value, noise_add.in2)
    yn = observed_variable("[$index] yn", noise_add.out)
    
    return (yn, noise_add)
end

function createGraph(size::Int)
    gmv_0   = gaussian_mean_variance("[0] gmv");
    gmv_0_m = estimated_variable("[0] gmv_m", gmv_0.mean);
    gmv_0_v = estimated_variable("[0] gmv_v", gmv_0.variance);
    
    add_c_0 = addition_node("[0] add_c", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
    x0 = random_variable("[0] x", gmv_0.value, add_c_0.in1)
    c0 = constant_variable("[0] c", 1.0, add_c_0.in2)
    
    index = 1
    
    prev_add_c = add_c_0
    
    xs = Vector{RandomVariable}()
    ys = Vector{ObservedVariable}()
    
    while index < size
        equality_n    = equality_ioo("[$index] equality")
        yn, noise_add = createSubgraph(index)
        
        xn  = random_variable("[$index] xn", prev_add_c.out, equality_n.in1)
        xn_ = random_variable("[$index] xn_", equality_n.out1, noise_add.in1)
        
        add_c = addition_node("[$index] add_c", StochasticMessage{Normal{Float64}}, DeterministicMessage, StochasticMessage{Normal{Float64}});
        
        xn__ = random_variable("[$index] xn__", equality_n.out2, add_c.in1)
        cn   = constant_variable("[$index] cn", 1.0, add_c.in2)
        
        push!(xs, xn)
        push!(ys, yn)
        
        prev_add_c = add_c
        
        index += 1
    end
    
    last_noise_add = addition_node("[last] noise_add", StochasticMessage{Normal{Float64}}, StochasticMessage{Normal{Float64}}, DeterministicMessage);
    x_last = random_variable("[last] x", prev_add_c.out, last_noise_add.in1)
    
    last_noise_node     = gaussian_mean_variance("[last] noise_node");
    last_noise_mean     = constant_variable("[last] noise_mean", 0.0, last_noise_node.mean);
    last_noise_variance = constant_variable("[last] noise_variance", 1.0, last_noise_node.variance);
    
    z      = random_variable("[last] z", last_noise_node.value, last_noise_add.in2)
    y_last = observed_variable("[last] y", last_noise_add.out)
    
    push!(xs, x_last)
    push!(ys, y_last)
    
    return (xs, ys, gmv_0_m, gmv_0_v)
end

createGraph (generic function with 1 method)

In [15]:
function smoothing() 
    N = 100
    data = collect(1:N) + rand(Normal(0.0, 1.0), N);
    
    xs, ys, prior_m, prior_v = createGraph(N);
    
    actor = sync(keep(Vector{AbstractMessage}))
    
    points = Vector{Any}(undef, N)
    for i in 1:N
        points[i] = inference(xs[i]) |> take(1)
    end
    
    subscribe!(collectLast(AbstractMessage, points), actor)
    
    update!(prior_m, 0.0)
    update!(prior_v, 1000.0)
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    wait(actor)
    
    return actor.actor.values[1]
end

smoothing (generic function with 1 method)

In [ ]:
v = smoothing()

In [14]:
map((m) -> mean(m.distribution), v)

ErrorException: type Array has no field distribution